## Discovering matrix multiplication algorithms

Here we illustrate how remarkable FedroTensor is at finding fast matrix multiplication algorithms.
Namely, our task is to minimise the number of scalar multiplications.
This problem is equivalent to finding the lowest possible CP rank of a certain 3-dimensional tensor.

Let's start with the 2x2 case. The naive approach results in 8 (=2x2x2) scalar multiplications, but there is a way to do it with only 7 multiplications, known as the Strassen algorithm.
Let's see if our methods can recover it.

In [22]:
from FedroTensor.tensor_rank import cp_rank
from FedroTensor.matmul_tensor import build_matmul_tensor

T = build_matmul_tensor(2, 2, 2)
r, factors = cp_rank(T)
r

7

The rank indeed corresponds to the Strassen algorithm cost. Let's see if we can recover the algorithm from the factors.

In [23]:
factors

[array([[ 0.6988722 , -1.15468194,  0.91949613, -0.22426491, -0.2524747 ,
          0.11743307,  0.75734964],
        [ 0.8833869 , -0.26718334, -0.13656764,  0.64163105, -0.31864572,
         -0.33412897, -0.11178165],
        [-0.10660735, -0.29681378, -0.13902532, -0.33043288,  0.77278801,
         -0.35755047,  1.1179974 ],
        [-0.13364547,  1.38626854,  0.02127065,  0.94621386,  0.97784532,
          1.02445013, -0.16502191]]),
 array([[ 0.03821822, -0.74008811, -0.20667224, -0.16313111, -1.14417988,
          0.62772695,  0.72965168],
        [ 0.16702723, -0.14546231, -0.90434511,  0.13017963, -0.38191673,
          0.20955392, -0.57827813],
        [ 0.25972843, -0.09706048,  0.16330807, -1.10543067, -0.39962335,
         -0.497342  ,  0.255311  ],
        [ 1.13130903,  0.65637623,  0.71487949,  0.87674152, -0.13326565,
         -0.16520449, -0.20166921]]),
 array([[ 0.63317248,  1.13509574,  0.35301237, -0.84984927, -0.08744625,
          1.46891907,  0.05295654],
      

Not helpful indeed...

Ideally, we would like to have integer factors, or at least the rational ones. We can compute them by specifying `rational=True`:

In [16]:
r, factors = cp_rank(T, rational=True)
factors

[array([[ 0.,  0.,  0.,  1.,  0.,  0., -1.],
        [-1., -1.,  0.,  0.,  0.,  1.,  1.],
        [ 0.,  1., -1., -1.,  1.,  0.,  0.],
        [ 1.,  0.,  0.,  0., -1.,  0.,  0.]]),
 array([[ 0.,  0.,  1.,  1.,  0.,  0.,  0.],
        [ 0., -1.,  0., -1.,  0.,  1., -1.],
        [ 1., -1.,  1.,  0., -1.,  0.,  0.],
        [-1.,  0.,  0.,  0.,  0.,  1.,  0.]]),
 array([[ 0.,  1., -1.,  1.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  1.],
        [ 0.,  0., -1.,  0.,  1.,  0.,  0.],
        [-1., -1.,  0.,  0.,  1.,  1.,  0.]])]

That's much better, as we can now write down the algorithm precisely.

Let's consider the 3x3 case. The best known algorithm (the Laderman's algorithm) has 23 multiplications. Remarkably, our method manages to find rank 23:

In [17]:
T = build_matmul_tensor(3, 3, 3)
r, factors = cp_rank(T)
r

23

Computing rational factors takes slightly longer, but it's definitely worth it:

In [19]:
r, factors = cp_rank(T, rational=True)
factors

[array([[ 0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.],
        [ 1.,  1.,  0., -1.,  0.,  0.,  0.,  0., -1., -1., -1.,  0.,  0.,
          0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0., -1., -1.,  0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0., -1.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -1.,
          0.,  0.,  0., -1., -1.,  0., -1.,  0.,  0., -1.],
        [ 0.,  0.,  0.,  0.,  1.,  0., -1.,  1.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  0.,  0.

Finally, it's time for a benchmark on different sizes (n, m, p).

In [21]:
sizes = [(2, 2, 2), (2, 2, 3), (2, 2, 4), (2, 2, 5), (2, 3, 3), (2, 3, 4), (2, 3, 5), (2, 4, 4), (2, 4, 5), (2, 5, 5), (3, 3, 3), (3, 3, 4), (3, 3, 5), (3, 4, 4), (3, 4, 5), (4, 4, 4)]
true_rs = [7, 11, 14, 18, 15, 20, 25, 26, 33, 40, 23, 29, 36, 38, 47, 49]
print('n\tm\tp\tr\ttrue_r')
for (n, m, p), true_r in zip(sizes, true_rs):
    T = build_matmul_tensor(n, m, p)
    r, _ = cp_rank(T)
    print(f'{n}\t{m}\t{p}\t{r}\t{true_r}')

n	m	p	r	true_r
2	2	2	7	7
2	2	3	11	11
2	2	4	14	14
2	2	5	18	18
2	3	3	15	15
2	3	4	20	20
2	3	5	26	25
2	4	4	26	26
2	4	5	34	33
2	5	5	41	40
3	3	3	23	23
3	3	4	31	29
3	3	5	39	36
3	4	4	42	38
3	4	5	52	47
4	4	4	56	49


Notice our method finds optimal ranks when the rank is less than 25. In the larger cases, it's slightly worse (for now). One can increase `num_attempts` from 10 to, say, 100, but then the computation becomes proportionally slower.